In [1]:
import pandas
import torch

from transformers import AutoModel, AutoTokenizer

In [2]:
PATH = '/mnt/hlilabshare/jjunshim/data/situated_qa/situated_qa_sample_id100.parquet'
df = pandas.read_parquet(PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         300 non-null    int64         
 1   timestamp  300 non-null    datetime64[ns]
 2   text       300 non-null    object        
 3   title      300 non-null    object        
 4   infobox    300 non-null    object        
 5   wikitable  300 non-null    object        
 6   snap       300 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 16.5+ KB


In [2]:
PATH = '/mnt/hlilabshare/jjunshim/data/temporal-alignment-qa/sample.parquet'
df = pandas.read_parquet(PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      24 non-null     int64 
 1   year    24 non-null     object
 2   text    24 non-null     object
dtypes: int64(1), object(2)
memory usage: 708.0+ bytes


# Model

## TPO model

In [3]:
PATH = '/mnt/hlilabshare/jjunshim/models/tpo/2018/checkpoint.pth'
tokenizer = AutoTokenizer.from_pretrained("facebook/contriever")
model = AutoModel.from_pretrained(
    'bert-base-uncased',
    PATH
)

# model.to('cuda')
print(model.device, model)

cpu BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=F

In [4]:
tokens = tokenizer(
    list(df.loc[:, 'text'].values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

with torch.no_grad():
    outputs = model(
        **tokens,
        output_hidden_states=True,
        # output_attentions=True
    )

# torch.cuda.empty_cache()

In [5]:
model = None
tokenizer = None
tokens = None

In [ ]:
torch.save(
    outputs.__dict__,
    '/mnt/hlilabshare/jjunshim/models/outputs_tpo_2018.pth'
)

In [6]:
torch.save(
    outputs.__dict__,
    '/mnt/hlilabshare/jjunshim/models/outputs_tpo_2018_sample.pth'
)

## Contriever

In [7]:
tokenizer = AutoTokenizer.from_pretrained("facebook/contriever")
model = AutoModel.from_pretrained(
    'bert-base-uncased',
    'google-bert/bert-base-uncased'
    # 'facebook/contriever'
)
tokens = tokenizer(
    list(df.loc[:, 'text'].values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

with torch.no_grad():
    outputs = model(
        **tokens,
        output_hidden_states=True,
        # output_attentions=True
    )

model = None
tokenizer = None
tokens = None

del model
del tokenizer
del tokens

torch.save(
    outputs.__dict__,
    '/mnt/hlilabshare/jjunshim/models/outputs_bert_sample.pth'
    # '/mnt/hlilabshare/jjunshim/models/outputs_contriever_sample.pth'
)

In [2]:
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "image-text-to-text",
    model="google/gemma-3-27b-it",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-27b-it")


model.safetensors.index.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

model-00001-of-00012.safetensors:   0%|          | 0.00/4.85G [00:00<?, ?B/s]

model-00002-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00004-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

model-00007-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00008-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00009-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00010-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00011-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer(
    list(df.loc[:, 'text'].values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
with torch.no_grad():
    outputs = model(
        **tokens,
        output_hidden_states=True,
        output_attentions=True
    )
torch.save(
    outputs.__dict__,
    '/mnt/hlilabshare/jjunshim/models/google/gemma-3-27b-it_sample.pth'
)